## Import Libraries

In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import joblib
import time

In [18]:
start_time = time.perf_counter()

## Configuration

In [19]:
# Configuration
MODEL_FILEPATH = './models/credit_score_regressor_ipynb.joblib'
DATA_FILEPATH = './data/train.csv'
NUM_TREES = 10000
LEARNING_RATE = 0.01
EARLY_STOPPING_ROUNDS = 10

## Define Columns to drop & Columns contain Numerical Features 

In [20]:
# Columns to drop
columns_to_drop = [
    'ID',
    'Customer_ID',
    'Month',
    'Name',
    'SSN',
    'Type_of_Loan'
]

# Numerical features
numerical_features = [
    'Age',
    'Annual_Income',
    'Num_Bank_Accounts',
    'Num_Credit_Card',
    'Interest_Rate',
    'Delay_from_due_date',
    'Num_of_Delayed_Payment',
    'Changed_Credit_Limit',
    'Num_Credit_Inquiries',
    'Outstanding_Debt',
    'Credit_Utilization_Ratio',
    'Total_EMI_per_month',
    'Credit_History_Months',
    'Credit_Mix_Encoded'
]

## Load Data From CSV & Drop Unnecessary Columns

In [21]:
# Load data
print("Loading data...")
df = pd.read_csv(DATA_FILEPATH)
print(f"Original shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst 5 rows:")
print(df.head().to_string(index=False))
df_dropped = df.drop(columns=columns_to_drop, axis=1)
df = df_dropped.copy()
print(f"Shape after dropping: {df.shape}")
print(f"Remaining columns: {list(df.columns)}")
print(df.head().to_string(index=False))


Loading data...


Original shape: (100000, 28)
Columns: ['ID', 'Customer_ID', 'Month', 'Name', 'Age', 'SSN', 'Occupation', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts', 'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan', 'Type_of_Loan', 'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit', 'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt', 'Credit_Utilization_Ratio', 'Credit_History_Age', 'Payment_of_Min_Amount', 'Total_EMI_per_month', 'Amount_invested_monthly', 'Payment_Behaviour', 'Monthly_Balance', 'Credit_Score']

First 5 rows:
    ID Customer_ID    Month          Name  Age         SSN Occupation Annual_Income  Monthly_Inhand_Salary  Num_Bank_Accounts  Num_Credit_Card  Interest_Rate Num_of_Loan                                                        Type_of_Loan  Delay_from_due_date Num_of_Delayed_Payment Changed_Credit_Limit  Num_Credit_Inquiries Credit_Mix Outstanding_Debt  Credit_Utilization_Ratio    Credit_History_Age Payment_of_Min_Amount  Total_EMI_per_

In [22]:
print("Data types:")
print(df.dtypes)
print("\nMissing values:")
print(df.isnull().sum())

Data types:
Age                          object
Occupation                   object
Annual_Income                object
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                  object
Delay_from_due_date           int64
Num_of_Delayed_Payment       object
Changed_Credit_Limit         object
Num_Credit_Inquiries        float64
Credit_Mix                   object
Outstanding_Debt             object
Credit_Utilization_Ratio    float64
Credit_History_Age           object
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly      object
Payment_Behaviour            object
Monthly_Balance              object
Credit_Score                 object
dtype: object

Missing values:
Age                             0
Occupation                      0
Annual_Income                   0
Monthly_Inhand_Salary       15002
Num_Bank_Accounts            

## Clean Data
### Clean numerical columns

In [23]:
# Clean numerical columns
print("Cleaning data...")
cols_to_clean = ['Age', 'Annual_Income', 'Outstanding_Debt', 'Changed_Credit_Limit', 'Num_of_Delayed_Payment']
for col in cols_to_clean:
    if df[col].dtype == 'object':
        df[col] = df[col].replace({'_': np.nan, '!!': np.nan, '#': np.nan}, regex=True)
        df[col] = pd.to_numeric(df[col], errors='coerce')

Cleaning data...


### Convert Credit History Age to months

In [24]:
# Convert Credit History Age to months
def convert_age_to_months(age_str):
    if pd.isna(age_str):
        return np.nan
    try:
        parts = age_str.split(' and ')
        years = int(parts[0].split(' ')[0])
        months = int(parts[1].split(' ')[0])
        return years * 12 + months
    except:
        return np.nan
    
df['Credit_History_Months'] = df['Credit_History_Age'].apply(convert_age_to_months)

# Show before and after (First 10 rows)
comparison_df = df[['Credit_History_Age', 'Credit_History_Months']].head(10)
print(comparison_df.to_string(index=False))


   Credit_History_Age  Credit_History_Months
22 Years and 1 Months                  265.0
                  NaN                    NaN
22 Years and 3 Months                  267.0
22 Years and 4 Months                  268.0
22 Years and 5 Months                  269.0
22 Years and 6 Months                  270.0
22 Years and 7 Months                  271.0
                  NaN                    NaN
26 Years and 7 Months                  319.0
26 Years and 8 Months                  320.0


### Encode Credit Mix

In [ ]:
# Encode Credit_Mix
df['Credit_Mix'] = df['Credit_Mix'].replace({'_': np.nan})
le = LabelEncoder()
non_null_mix = df['Credit_Mix'].dropna().unique()
le.fit(non_null_mix)
df.loc[df['Credit_Mix'].notna(), 'Credit_Mix_Encoded'] = le.transform(df.loc[df['Credit_Mix'].notna(), 'Credit_Mix'])

# Show before and after (First 10 rows)
comparison_df = df[['Credit_Mix', 'Credit_Mix_Encoded']].head(10)
print(comparison_df.to_string(index=False))

### Fill missing value with mean

In [26]:
# Fill missing values with mean
for col in numerical_features:
    df[col] = df[col].fillna(df[col].mean())

### Convert categorical credit scores to numerical
Due to the lack of numerical data in the dataset, we assume the numerical score of each row like the following:
- 'Poor' = (300 + 580)/2 = 440
- 'Standard' = (580 + 740)/2 = 660
- 'Good' = (670 + 851)/2 = 760.

In [27]:
# Convert categorical credit scores to numerical (FICO-like scale: 300-850)
print("Converting credit scores to numerical values...")
df['Numerical_Credit_Score'] = df['Credit_Score'].apply(
    lambda x: int((300 + 580)/2) if x == 'Poor' 
    else int((580 + 740)/2) if x == 'Standard'
    else int((670 + 851)/2) if x == 'Good'
    else 500  # default
)

# Show before and after (First 10 rows)
comparison_df = df[['Credit_Score', 'Numerical_Credit_Score']].head(10)
print(comparison_df.to_string(index=False))

Converting credit scores to numerical values...
Credit_Score  Numerical_Credit_Score
        Good                     760
        Good                     760
        Good                     760
        Good                     760
        Good                     760
        Good                     760
        Good                     760
    Standard                     660
    Standard                     660
        Good                     760


### Split Data
Device columns to determine which columns using for evaluating and which columns the desired result.

In [28]:
X = df[numerical_features]
y = df['Numerical_Credit_Score']

print(f"Dataset shape: {X.shape}")
print(f"Credit score range: {y.min()} - {y.max()}")
print(f"Credit score mean: {y.mean():.2f}")

Dataset shape: (100000, 14)
Credit score range: 440 - 760
Credit score mean: 614.03


Split data horizontally to use a part of the data for trainning and the rest for testing.

In [29]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
)

## Training Model
### Set parameters for XGBoost Regression Model

In [30]:
# Initialize XGBoost Regressor
print("\nInitializing XGBoost Regressor...")
xgb_model = XGBRegressor(
    objective='reg:squarederror',    # Regression objective
    n_estimators=NUM_TREES,
    learning_rate=LEARNING_RATE,
    max_depth=10,
    min_child_weight=1,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric='rmse',
    random_state=404,
    early_stopping_rounds=EARLY_STOPPING_ROUNDS,
    tree_method='hist'
)


Initializing XGBoost Regressor...


### Start training the model

In [31]:
# Train model
eval_set = [(X_train, y_train), (X_test, y_test)]

print("Starting XGBoost model training...")
xgb_model.fit(
    X_train, 
    y_train,
    eval_set=eval_set,
    verbose=True  # Show progress
)
print("\nModel training complete.")

Starting XGBoost model training...
[0]	validation_0-rmse:116.39455	validation_1-rmse:116.93092
[1]	validation_0-rmse:115.92254	validation_1-rmse:116.48386
[2]	validation_0-rmse:115.44283	validation_1-rmse:116.03551
[3]	validation_0-rmse:114.95450	validation_1-rmse:115.57459
[4]	validation_0-rmse:114.48319	validation_1-rmse:115.12991
[5]	validation_0-rmse:114.04898	validation_1-rmse:114.72928
[6]	validation_0-rmse:113.55869	validation_1-rmse:114.26973
[7]	validation_0-rmse:113.08453	validation_1-rmse:113.82803
[8]	validation_0-rmse:112.64943	validation_1-rmse:113.42399
[9]	validation_0-rmse:112.18200	validation_1-rmse:112.98440
[10]	validation_0-rmse:111.75478	validation_1-rmse:112.58506
[11]	validation_0-rmse:111.29821	validation_1-rmse:112.16120
[12]	validation_0-rmse:110.85798	validation_1-rmse:111.75255
[13]	validation_0-rmse:110.41283	validation_1-rmse:111.34327
[14]	validation_0-rmse:109.97125	validation_1-rmse:110.93148
[15]	validation_0-rmse:109.53929	validation_1-rmse:110.53105

### Save model for future use

In [32]:
try: 
    # Save model
    print(f"\nSaving model to {MODEL_FILEPATH}...")
    joblib.dump(xgb_model, MODEL_FILEPATH)
    print("Model saved successfully!")
except Exception as e:
    print(e)


Saving model to ./models/credit_score_regressor_ipynb.joblib...
Model saved successfully!


## Evaluating Model

In [33]:
print(f'Best score (RMSE): {xgb_model.best_score:.4f}')
print(f'Best iteration: {xgb_model.best_iteration}')

# Make predictions
print("\nEvaluating model...")
y_pred_train = xgb_model.predict(X_train)
y_pred_test = xgb_model.predict(X_test)

# Clip predictions to valid credit score range
y_pred_train = np.clip(y_pred_train, 300, 850)
y_pred_test = np.clip(y_pred_test, 300, 850)

# Calculate metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
train_mae = mean_absolute_error(y_train, y_pred_train)
test_mae = mean_absolute_error(y_test, y_pred_test)
train_r2 = r2_score(y_train, y_pred_train)
test_r2 = r2_score(y_test, y_pred_test)

print("\n" + "="*60)
print("MODEL PERFORMANCE")
print("="*60)
print(f"\nTraining Set:")
print(f"  RMSE: {train_rmse:.2f} points")
print(f"  MAE:  {train_mae:.2f} points")
print(f"  R²:   {train_r2:.4f}")

print(f"\nTest Set:")
print(f"  RMSE: {test_rmse:.2f} points")
print(f"  MAE:  {test_mae:.2f} points")
print(f"  R²:   {test_r2:.4f}")

# Feature importance
print("\n" + "="*60)
print("TOP 10 MOST IMPORTANT FEATURES")
print("="*60)
feature_importance = pd.DataFrame({
    'Feature': numerical_features,
    'Importance': xgb_model.feature_importances_
}).sort_values('Importance', ascending=False)

print(feature_importance.head(10).to_string(index=False))

# Sample predictions
print("\n" + "="*60)
print("SAMPLE PREDICTIONS (First 10 test samples)")
print("="*60)
sample_df = pd.DataFrame({
    'Actual': y_test.values[:10],
    'Predicted': y_pred_test[:10],
    'Difference': y_test.values[:10] - y_pred_test[:10]
})
print(sample_df.to_string(index=False))

Best score (RMSE): 73.0153
Best iteration: 3920

Evaluating model...

MODEL PERFORMANCE

Training Set:
  RMSE: 25.29 points
  MAE:  16.73 points
  R²:   0.9532

Test Set:
  RMSE: 73.02 points
  MAE:  50.99 points
  R²:   0.6133

TOP 10 MOST IMPORTANT FEATURES
              Feature  Importance
     Outstanding_Debt    0.222976
   Credit_Mix_Encoded    0.129271
        Interest_Rate    0.109520
      Num_Credit_Card    0.067940
 Num_Credit_Inquiries    0.061420
  Delay_from_due_date    0.059864
  Total_EMI_per_month    0.058027
 Changed_Credit_Limit    0.055088
    Num_Bank_Accounts    0.050527
Credit_History_Months    0.047247

SAMPLE PREDICTIONS (First 10 test samples)
 Actual  Predicted  Difference
    760 719.879639   40.120361
    440 483.766174  -43.766174
    760 711.938538   48.061462
    440 440.302979   -0.302979
    760 723.046936   36.953064
    440 446.353882   -6.353882
    760 720.237976   39.762024
    440 522.200012  -82.200012
    760 678.554932   81.445068
    660 679.

## Using the model

In [34]:
# Function to predict credit score for new data
def predict_credit_score(model_path, features_dict):
    model = joblib.load(model_path)
    features_df = pd.DataFrame([features_dict])
    prediction = model.predict(features_df)[0]
    return np.clip(prediction, 300, 850)

In [35]:
# Example usage (uncomment to use):
new_customer = {
    'Age': 35,
    'Annual_Income': 75000,
    'Num_Bank_Accounts': 3,
    'Num_Credit_Card': 2,
    'Interest_Rate': 12,
    'Delay_from_due_date': 5,
    'Num_of_Delayed_Payment': 2,
    'Changed_Credit_Limit': 5000,
    'Num_Credit_Inquiries': 3,
    'Outstanding_Debt': 15000,
    'Credit_Utilization_Ratio': 25,
    'Total_EMI_per_month': 1200,
    'Credit_History_Months': 120,
    'Credit_Mix_Encoded': 1
}

In [36]:
score = predict_credit_score(MODEL_FILEPATH, new_customer)
print(f"Predicted Credit Score: {score:.0f}")

Predicted Credit Score: 640


In [37]:
# Execution time
end_time = time.perf_counter()
execution_time = end_time - start_time
print(f"\nTotal execution time: {execution_time:.2f} seconds")


Total execution time: 415.86 seconds
